In [12]:
import pandas as pd
import matplotlib.pyplot as plt

# Bring in data

In [13]:
ihme_path = 'data/ihme-covid19-04072020/2020_04_05.08.all/Hospitalization_all_locs.csv'
ihme = pd.read_csv(ihme_path)

# Organize data

In [34]:
# IHME
ihme_mt = ihme[ihme['location_name'] == 'Montana']
ihme_mt.set_index('date', inplace=True)
allbed_cols = ['allbed_lower', 'allbed_mean', 'allbed_upper']
totdea_cols = ['totdea_lower', 'totdea_mean', 'totdea_upper']
deaths_cols = ['deaths_lower', 'deaths_mean', 'deaths_upper']
ICUbed_cols = ['ICUbed_lower', 'ICUbed_mean', 'ICUbed_upper']
ihme_allbed = ihme_mt[allbed_cols]
ihme_totdea = ihme_mt[totdea_cols]
ihme_deaths = ihme_mt[deaths_cols]
ihme_ICUbed = ihme_mt[ICUbed_cols]

# Calculate stats

In [35]:
# dop = day of peak
ihme_allbed_dop_lower, ihme_allbed_dop_mean, ihme_allbed_dop_upper = ihme_allbed.idxmax()
ihme_deaths_dop_lower, ihme_deaths_dop_mean, ihme_deaths_dop_upper = ihme_deaths.idxmax()
ihme_ICUbed_dop_lower, ihme_ICUbed_dop_mean, ihme_ICUbed_dop_upper = ihme_ICUbed.idxmax()

# max
ihme_allbed_max_lower, ihme_allbed_max_mean, ihme_allbed_max_upper = ihme_allbed.max(axis=0)
ihme_deaths_max_lower, ihme_deaths_max_mean, ihme_deaths_max_upper = ihme_deaths.max(axis=0)
ihme_ICUbed_max_lower, ihme_ICUbed_max_mean, ihme_ICUbed_max_upper = ihme_ICUbed.max(axis=0)





'2020-03-30'